# Step01: Filtering and processing newspaper titles

This is step 01 of the **PressPicker** tool.

In this notebook, we read in the title-level British Library newspaper dataset https://doi.org/10.23636/1136, filter it by date, geography, and to only undigitised titles. 

We process free-text data describing title name changes through time to create structured data.

We read in item-level data for microfilm and hardcopy holdings of these titles, re-form this data into time series, and join it to the title-level data.

In [ ]:
import numpy as np
import os
import pandas as pd
import re
import string

In [ ]:
# Always show all columns when displaying dataframes
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'

### Read in newspaper titles

In [ ]:
# read in newspaper titles spreadsheet from path2newspaper_titles
# This file is publically available at https://doi.org/10.23636/1136
path2newspaper_titles = os.path.join("datasets", "BritishAndIrishNewspapers_20191118.xlsx")
sheet_name = "Title List"

titles_orig_read = pd.read_excel(path2newspaper_titles, sheet_name=sheet_name, dtype='str')
titles = titles_orig_read
titles.head()
len(titles)

### Filtering newspaper titles

In [ ]:
# --- remove already digitised titles
# Online status: if BNA or BURNEY, they are already digitised and no need to keep them in the dataframe
print(titles["Online status"].value_counts())
print("-------")
print("Total number of titles: %s" % len(titles.index))
titles = titles[pd.isnull(titles["Online status"]) & pd.isnull(titles["Link to British Newspaper Archive"])]
print("Total number of undigitised titles:  %s" % len(titles.index))

In [ ]:
# Clean strings when doing filtering 
def cleanuplist(input_list):
    input_list = [re.sub(r"\s*\|\s*", "|", x) if (not pd.isnull(x))
            else x for x in input_list]
    input_list = [re.sub(r"\s+", " ", x) if (not pd.isnull(x)) else x for x in input_list]

    return input_list

In [ ]:
# --- cleanup `Country of publication`
titles['Country of publication'] = cleanuplist(titles['Country of publication'])

In [ ]:
# Print unique values of `Country of publication`
print("================ Unique values")
print(titles['Country of publication'].unique())
print("\n\n================ Unique values and their counts")
print(titles['Country of publication'].value_counts())

In [ ]:
# List of countries to include
countries_include_list = [
    'England', 
    'Scotland',
    'Wales',
    'England|Wales',
    'England|Scotland',
    'England|United Kingdom',
    'United Kingdom',
    'Scotland|United Kingdom'
                         ]

In [ ]:
print("Total number of titles (before filtering): %s" % len(titles.index))
titles = titles[titles['Country of publication'].isin(countries_include_list)]
print("Total number of titles (after filtering):  %s" % len(titles.index))

## Correct dates and filter

In [ ]:
# Correct for dates in form '1895|1899', '1981|Continuing', 'Continuing'
# Assume: "continuing" means still being published, replace date with current_date
def correct_dates_titles(mydate, current_date=2021):
    if pd.notna(mydate):
        # if continu* is in mydate, replace it with current_date
        if "continu" in str(mydate).lower():
            mydate = current_date
        
        if "|" in str(mydate):
            return int(mydate.split("|")[0])
        else:
            return int(mydate)
        
    else:
        return np.nan

In [ ]:
# current_date is used to deal with "Continuing" fields in dates
current_date = 2021

titles['First date held'] = titles['First date held'].apply(correct_dates_titles, args=(current_date,))
titles['Last date held'] = titles['Last date held'].apply(correct_dates_titles, args=(current_date,))

In [ ]:
# filter by date
earliest_date_filter = 1780
latest_date_filter = 1918

# Filter where overlap with our date span, but EXCLUDE where no first or last date held
def filter_by_dates (df):
    return df[((df['First date held'] <= latest_date_filter)| pd.isnull(df['First date held'])) & 
              ((df['Last date held']  >= earliest_date_filter) | pd.isnull(df['Last date held'])) & 
              ~(pd.isnull(df['First date held']) & pd.isnull(df['Last date held']))]

print("Total number of titles (before filtering): %s" % len(titles))
# Filter titles by dates
titles = filter_by_dates(titles)
print("Total number of titles (after filtering): %s" % len(titles))

In [ ]:
titles = titles.reset_index(drop=True)

<!-- # Checking against Original Request -->

## Read Hardcopy

In [ ]:
# read in hardcopy spreadsheet
items_hardcopy_path = os.path.join("datasets", "titles_ALLnotDigitised_31_10_2019 - RESULTS.xlsx")
items_hardcopy = pd.read_excel(items_hardcopy_path, dtype='str')

# rename some columns
items_hardcopy.rename(columns={"ID": "ID_HC"}, inplace=True)

print("Total number of Hard Copies, after filtering: %s" % len(items_hardcopy.index))
items_hardcopy.head()

### Filter hardcopies

In [ ]:
# Remove rows where Sublibrary == HMNTS (these are positive microfilm - which are excluded in Press Picker)
items_hardcopy = items_hardcopy[items_hardcopy['sublibrary'] != 'HMNTS']
print("Total number of Hard Copies, after removing positive microfilms: %s" % len(items_hardcopy.index))
items_hardcopy.head()

In [ ]:
# ====== merge titles with HCs, we use Title.ID and ID_HC
titles_hardcopy = pd.merge(titles, items_hardcopy, left_on='Title.ID', right_on='ID_HC', how='outer')
print("Title's shape: {}".format(titles.shape))
print("HC shape: {}".format(items_hardcopy.shape))
print("Title-HC merge: {}".format(titles_hardcopy.shape))
titles_hardcopy

In [ ]:
# Title.ID can be NaN since some of the titles have been already filtered out (see the cells above) - remove these rows
titles_hardcopy = titles_hardcopy[~pd.isnull(titles_hardcopy['Title.ID'])]
print("Title-HC merge, after filtering: {}".format(titles_hardcopy.shape))

## Read Microfilm

In [ ]:
# read in microfilm spreadsheet
items_microfilm_path = os.path.join("datasets", "nmf-items-1911-0320.xlsx")
items_microfilm = pd.read_excel(items_microfilm_path,
                                sheet_name='Sheet3',
                                dtype='str')

items_microfilm = items_microfilm[~pd.isnull(items_microfilm['bib'])]
print("MF shape: {}".format(items_microfilm.shape))
items_microfilm.head()
print("Filter out titles with no canNo.")
items_microfilm = items_microfilm[~pd.isnull(items_microfilm['canNo'])]
print("MF shape: {}".format(items_microfilm.shape))

In [ ]:
# Microfilm can numbers are all numbers except for '0180B' which causes filtering problems
# since item '0180B' is out of our date range (1952), exclude it
items_microfilm = items_microfilm[items_microfilm['canNo'] != '0180B']

In [ ]:
# ====== merge titles with MFs, we use Title.ID and bib
titles_mf = pd.merge(titles, items_microfilm, left_on='Title.ID', right_on='bib', how='outer')
print("Title-MF merge: {}".format(titles_mf.shape))

In [ ]:
# Title.ID can be NaN since some of the titles have been already filtered out (see the cells above) - remove these
titles_mf = titles_mf[~pd.isnull(titles_mf['Title.ID'])]
print("Title-MF merge, after filtering: {}".format(titles_mf.shape))

In [ ]:
# Change canNo column to numbers, so can filter on this later
titles_mf['canNo'] = titles_mf['canNo'].astype('float')

In [ ]:
titles_mf.head()

## Create "connectivity": titles storing the IDs of connected titles

In this section, we create structured data to connect titles that are related by name changes through time. 

**Steps:**
1. Preprocess `Publication title`, `Preceding titles`, `Succeeding titles` and `General area of coverage` using `clean_succeed_preceding` function (see below). Note that, in the first three cases (i.e., `Publication/Preceding/Succeeding titles`), if title(s) exist, they are converted into a Python list; otherwise, `nan` will be stored. 
2. Here, we find the `Title.ID`s of preceding/succeeding titles of each `Publication title`. We use the Python list created in Step 1 (and skip those that are `nan`). We only consider the preceding/succeeding titles with similar `general_area_of_coverage_str` to the main publication title. See for example:

```python
found_titles = titles[(titles["publication_title_str"] == ititle) & (titles["general_area_of_coverage_str"] == ititle_General_area_of_coverage)]
```

In [ ]:
# String matching between preceding/succeeding titles and publication titles
# Introduces new 'connectivity' facet for titles storing the IDs of connected titles
def clean_succeed_preceding(mytitle, makelist=True):
    if 'nan' in str(mytitle):
        return "nan"
    mytitle = str(mytitle).lower()
    mytitle = re.sub("no\..*", "", mytitle)
    mytitle = re.sub("vol\..*", "", mytitle)
    mytitle = re.sub("[0-9].*", "", mytitle)
    # Strip out 'the's
    mytitle = re.sub("(^|\s)the($|\s)", "", mytitle, flags=re.IGNORECASE)
    mytitle = re.sub(r'Continued by:', '', mytitle, flags=re.IGNORECASE)
    mytitle = re.sub(r'Continued in part by:', '', mytitle, flags=re.IGNORECASE)
    mytitle = re.sub(r'continues:', '', mytitle, flags=re.IGNORECASE)
    mytitle = re.sub(r'continues in part:', '', mytitle, flags=re.IGNORECASE)
    mytitle = re.sub(re.compile(r'\s+'), '', mytitle)
    mytitle = re.sub(re.compile(r' \(.*\)'), '', mytitle)
   
    if str(mytitle) == '':
        return "nan"
    elif makelist:
        mytitle = re.sub('[%s]' % re.escape(string.punctuation.replace("|", "")), '', mytitle)
        return mytitle.split("|")
    else:
        mytitle = re.sub('[%s]' % re.escape(string.punctuation.replace("|", "")), '', mytitle)
        return mytitle

titles["connectivity"] = titles["Title.ID"]
titles["connectivity"] = ''
 
titles["publication_title_str"] = titles["Publication title"]
titles["preceding_title_str"] = titles["Preceding titles"]
titles["succeeding_title_str"] = titles["Succeeding titles"]
 
titles["publication_title_str"] = titles["publication_title_str"].apply(clean_succeed_preceding, makelist=False)
titles["preceding_title_str"] = titles["preceding_title_str"].apply(clean_succeed_preceding, makelist=True)
titles["succeeding_title_str"] = titles["succeeding_title_str"].apply(clean_succeed_preceding, makelist=True)
 
titles["general_area_of_coverage_str"] = titles["General area of coverage"].apply(clean_succeed_preceding, makelist=False)

In [ ]:
for irow in range(len(titles)):
    connectivity = []       
    if not titles["preceding_title_str"][irow] == 'nan':
        for ititle in titles["preceding_title_str"][irow]:
            ititle_General_area_of_coverage = titles["general_area_of_coverage_str"][irow]
            # checking if title matches have the same General.area.of.coverage
            found_titles = titles[(titles["publication_title_str"] == ititle) & (titles["general_area_of_coverage_str"] == ititle_General_area_of_coverage)]
            if len(found_titles) > 0:
                connectivity.extend(found_titles["Title.ID"].values.tolist())
    if not titles["succeeding_title_str"][irow] == 'nan':
        for ititle in titles["succeeding_title_str"][irow]:
            ititle_General_area_of_coverage = titles["general_area_of_coverage_str"][irow]
            found_titles = titles[(titles["publication_title_str"] == ititle) & (titles["general_area_of_coverage_str"] == ititle_General_area_of_coverage)]
            if len(found_titles) > 0:
                connectivity.extend(found_titles["Title.ID"].values.tolist())
    if len(connectivity) > 0:
        connectivity = list(dict.fromkeys(connectivity))
        titles.at[irow, "connectivity"] = ",".join(connectivity)
    else:
        titles.at[irow, "connectivity"] = np.nan

<!-- ## Sanity check, Number of HCs and Mfs -->

## Timeseries dataframe

Create timeseries data from the microfilm and hardcopy holdings for visualising in Step 02

In [ ]:
# Create new timeseries dataframe for items
earliest_date_titles = pd.to_numeric(titles['First date held']).min()
latest_date_titles = pd.to_numeric(titles['Last date held']).max()

full_date_span = list(range(int(earliest_date_titles), int(latest_date_titles) + 10))

timeseries_items = pd.DataFrame(columns=full_date_span)
# add empty column at end for items with unknown dates
timeseries_items["Total_canNos_below_4000"] = ""

# HC: Hard Copy, MF: Microfilm
timeseries_items_hc = timeseries_items.copy()
timeseries_items_mf = timeseries_items.copy()
timeseries_items_hc.head()

In [ ]:
def correct_dates_001(mydate):
    if 'nan' in str(mydate):
        return "Unknown"
    elif "|" in str(mydate):
        return mydate.split("|")[0]
    elif "/" in str(mydate):
        return mydate.split("/")[0]
    else:
        return mydate[:4]

### Timeseries for Hardcopy

In [ ]:
# Fill timeseries_items_hc dataframe
unique_title_hc = titles_hardcopy['ID_HC'].unique() 
unique_title_hc = unique_title_hc[~pd.isna(unique_title_hc)]

id_hc_count = []
for irow in unique_title_hc:
    titles_hardcopy_row = titles_hardcopy[titles_hardcopy['ID_HC'] == irow]
    id_hc_count.append([irow, 
                        len(titles_hardcopy_row), 
                        titles_hardcopy_row['Chron I'].apply(correct_dates_001).to_list()])
    timeseries_items_hc.at[irow, :] = 0
    for idate in id_hc_count[-1][-1]:
        if idate == 'Unknown':
            pass
        elif len(idate) > 4:
            idate = idate.replace(" ", "")
            first_date_hc = int(idate[:4])
            last_date_hc = int(idate[-4:])
            year_difference = last_date_hc - first_date_hc
            fraction_per_year = 1. / (year_difference + 1.) # If hc item covers a time span, timeseries is populated with fraction per year
            for iyear in range(first_date_hc, last_date_hc + 1):
                timeseries_items_hc.at[irow, iyear] += fraction_per_year
        else:
            timeseries_items_hc.at[irow, int(idate)] += 1

In [ ]:
timeseries_items_hc.head()

In [ ]:
columns_to_keep_names = list(range(int(earliest_date_filter), int(latest_date_filter + 1)))
columns_to_keep_names.append('Total_canNos_below_4000')
timeseries_items_hc = timeseries_items_hc.loc[:,columns_to_keep_names]
timeseries_items_hc.head()

In [ ]:
# sort timeseries_items_hc based on #HCs
idx_hc = timeseries_items_hc.sum(axis=1).sort_values(ascending=False).index
timeseries_items_hc = timeseries_items_hc.loc[idx_hc]
timeseries_items_hc.head()

### Timeseries for Microfilms
Create timeseries data for microfilms and create count of acetate microfilms (important to know for digitisation, and and an element of the visualisation in Step 02)

In [ ]:
# Fill timeseries_items_mf dataframe
# Microfilm records can cover a date span (between 'startDate' and 'endDate'): splits the volume count (1) over the date span 
# Excludes records with unknown dates (these also have no Can location information) - these are notes in Aleph database rather than referring to physcial resources
unique_title_mf = titles_mf['bib'].unique() 
unique_title_mf = unique_title_mf[~pd.isna(unique_title_mf)]

bib_mf_count = []
# BL microfilms with can number below 4000 are likely made of acetate. This is important to know for digitisation, so the count in recorded for each title. The count is visualised in Step 02.
can_acetate_threshold = 4000
for irow in unique_title_mf:
    titles_mf_row = titles_mf[titles_mf['bib'] == irow]
    bib_mf_count.append([irow, 
                         len(titles_mf_row), 
                         titles_mf_row[['startDate', 'endDate']].applymap(correct_dates_001).values.tolist()])
    timeseries_items_mf.at[irow, :] = 0
        
    # Only count'Total_canNos_below_4000' for records between 1780 - 1918
    if not titles_mf_row[(titles_mf_row['canNo'] < can_acetate_threshold)][['startDate']].applymap(correct_dates_001).empty:
        startDates_for_acetate_records = titles_mf_row[(titles_mf_row['canNo'] < can_acetate_threshold)][['startDate']].applymap(correct_dates_001)
        timeseries_items_mf.at[irow, f"Total_canNos_below_{can_acetate_threshold}"] += len(startDates_for_acetate_records[(startDates_for_acetate_records['startDate'].astype(int) >= int(earliest_date_filter)) & (startDates_for_acetate_records['startDate'].astype(int) <= int(latest_date_filter))])
    
    for idate in bib_mf_count[-1][-1]:
        if idate[0] == 'Unknown':
            pass
        elif (idate[1] == 'Unknown') | (idate[0] == idate[1]) :
            timeseries_items_mf.at[irow, int(idate[0])] += 1
        else:
            year_difference = int(idate[1]) - int(idate[0])
            fraction_per_year = 1. / (year_difference + 1.)
            for iyear in range(int(idate[0]), int(idate[1]) + 1):
                try:
                    timeseries_items_mf.at[irow, iyear] += fraction_per_year
                except:
                    # Throws an exception where date is outside date range, and therefore not covered by timeseries_items_mf
                    continue

In [ ]:
print(columns_to_keep_names)
timeseries_items_mf = timeseries_items_mf.loc[:,columns_to_keep_names]
timeseries_items_mf.head()

In [ ]:
# sort timeseries_items_mf based on #MFs
idx_bib_mf = timeseries_items_mf.sum(axis=1).sort_values(ascending=False).index
timeseries_items_mf = timeseries_items_mf.loc[idx_bib_mf]
timeseries_items_mf.head()

## Outputs

This cell exports datasets that are used in Step02 

In [ ]:
parent_path = os.path.join("datasets", "dynamic_io")

if not os.path.isdir(parent_path): 
    os.makedirs(parent_path)
    
titles.to_csv(os.path.join(parent_path, "titles.csv"))
timeseries_items_mf.to_csv(os.path.join(parent_path, "timeseries_items_mf.csv"))
timeseries_items_hc.to_csv(os.path.join(parent_path, "timeseries_items_hc.csv"))
titles_hardcopy.to_csv(os.path.join(parent_path, "titles_hc.csv"))
titles_mf.to_csv(os.path.join(parent_path, "titles_mf.csv"))